# Web Scraping

Los objetivos de aprendizaje son:

1. Extraer datos de sitios web
    - Crear un Scraper
    - Extraer datos con métodos string
2. BeautifulSoup
    - Clase BeautifulSoup
    - Métodos
3. Interactuar con formularios HTML
    - Browser
    - Enviar Formulario


## Extraer datos de sitios web


La recopilación de datos de sitios web mediante un proceso automatizado se conoce como web scraping. 

Algunos sitios web prohíben explícitamente que los usuarios extraigan sus datos con herramientas automatizadas.

Hacer muchas solicitudes repetidas al servidor de un sitio web puede consumir ancho de banda, ralentizar el sitio web para otros usuarios y potencialmente sobrecargar el servidor de modo que el sitio web deje de responder por completo.


> **Importante**: Las siguientes técnicas pueden ser ilegales cuando se utilizan en sitios web que prohíben el web scraping.


La página que usaremos en esta clase ha sido diseñada explícitamente para propósitos académicos en el área WEB.


### Crear un Scraper

`urllib` es un paquete útil para hacer web scraping, y forma parte de la librería standar de Python. 

En particular, el módulo `urllib.request` contiene una función llamada `urlopen()` que puedemos usar para abrir una URL dentro de un programa.

In [1]:
from urllib.request import urlopen
url = "http://olympus.realpython.org/profiles/aphrodite"
page = urlopen(url)

La función `urlopen()` regresa un objeto de la clase `HTTPResponse`.

Para extraer el HTML de la página debemos:

1. Usar el método `.read()` del objeto `HTTPResponse`, que devuelve una secuencia de bytes. 

2. Usar el método `.decode()` del objeto `bytes` para decodificar los bytes en una string usando UTF-8.

In [2]:
html_bytes = page.read()
html = html_bytes.decode("utf-8")
print(html)

<html>
<head>
<title>Profile: Aphrodite</title>
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/aphrodite.gif" />
<h2>Name: Aphrodite</h2>
<br><br>
Favorite animal: Dove
<br><br>
Favorite color: Red
<br><br>
Hometown: Mount Olympus
</center>
</body>
</html>



### Extraer datos con métodos string

Una forma de extraer información del HTML es usar los métodos de la clase string.

Supongamos que queremos extraer el texto `'Profile: Aphrodite'`, podríamos hacer lo siguiente:

In [3]:
def get_title(html: str)->str:
    title_index = html.find("<title>")
    start_index = title_index + len("<title>")
    end_index = html.find("</title>")
    title = html[start_index:end_index]
    return title

get_title(html)

'Profile: Aphrodite'

El HTML del mundo real puede ser mucho más complicado y mucho menos predecible que el HTML de la página que estamos usando, por ejemplo:

In [4]:
url = "http://olympus.realpython.org/profiles/poseidon"
page = urlopen(url)
html = page.read().decode("utf-8")
get_title(html)

'\n<head>\n<title >Profile: Poseidon'

In [5]:
print(html)

<html>
<head>
<title >Profile: Poseidon</title>
</head>
<body bgcolor="yellow">
<center>
<br><br>
<img src="/static/poseidon.jpg" />
<h2>Name: Poseidon</h2>
<br><br>
Favorite animal: Dolphin
<br><br>
Favorite color: Blue
<br><br>
Hometown: Sea
</center>
</body>
</html>



El HTML de la página `/profiles/poseidon` se parece a la página /`profiles/aphrodite`, pero hay una pequeña diferencia. El tag de apertura `<título>` tiene un espacio adicional antes del corchete angular de cierre (>).

Este tipo de problemas pueden ocurrir de innumerables formas impredecibles.

## BeautifulSoup

Es un paquete de Python diseñado para extraer datos de archivos HTML y XML.

### Clase BeautifulSoup

La siguiente celda de código hace 3 cosas:

1. Abre la URL http://olympus.realpython.org/profiles/dionysus usando `urlopen()`.
<br>

2. Lee el HTML de la página como una string y lo asigna a la variable html.
<br>

3. Crea un objeto `BeautifulSoup` y lo asigna a la variable de soup.

Para crear una instancia de la clase `BeautifulSoup` usamos dos argumentos:

- El primer argumento es el HTML que se va a analizar.
- El segundo argumento, la string "html.parser" que se usa para analizar documentos tipo HTLM.

In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = "http://olympus.realpython.org/profiles/dionysus"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
soup

<html>
<head>
<title>Profile: Dionysus</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<img src="/static/dionysus.jpg"/>
<h2>Name: Dionysus</h2>
<img src="/static/grapes.png"/><br/><br/>
Hometown: Mount Olympus
<br/><br/>
Favorite animal: Leopard <br/>
<br/>
Favorite Color: Wine
</center>
</body>
</html>

### Métodos

La clase `BeautifulSoup` cuenta con muchos método muy útiles para extraer texto, por ejemplo `.get_text()`:

In [6]:
print(soup.get_text())



Profile: Dionysus





Name: Dionysus

Hometown: Mount Olympus

Favorite animal: Leopard 

Favorite Color: Wine






Normalmente las etiquetas HTML son los elementos que señalan los datos que queremos extraer.

Por ejemplo, tal vez queremos recuperar las URL de todas las imágenes de la página. Estos enlaces están contenidos en el atributo src de las etiquetas `<img>`.

En este caso, podemos usar `find_all()` para devolver una lista de todas las instancias de una etiqueta:

In [7]:
soup.find_all("img")

[<img src="/static/dionysus.jpg"/>, <img src="/static/grapes.png"/>]

Esto devuelve una lista de todas las etiquetas `<img>` en el documento HTML.

Los objetos de la lista parecen strings, pero en realidad son instancias del objeto `Tag`. Esta clase proporcionan una interfaz sencilla para trabajar con la información que contienen.

In [8]:
img1 = soup.find_all("img")[0]
img1

<img src="/static/dionysus.jpg"/>

Podemos acceder a los atributos HTML del objeto `Tag` usando el método `.get()`.

Por ejemplo, el tag `<img src="/static/dionysus.jpg"/>` tiene un solo atributo, `src`, con el valor `"/static/dionysus.jpg".`.


In [9]:
img1.get('src')

'/static/dionysus.jpg'

Se puede acceder a ciertas etiquetas en documentos HTML mediante las propiedades del objeto `BeautifulSoup`. Por ejemplo, para obtener el tag `<title>`:

In [10]:
soup.title

<title>Profile: Dionysus</title>

In [11]:
soup.title.string

'Profile: Dionysus'

Una de las características de `BeautifulSoup` es la capacidad de buscar tipos específicos de tags cuyos atributos coincidan con ciertos valores.

Por ejemplo, podríamos buscar sólo los tags `<img>` que tengan un atributo `src` igual al valor `/static/dionysus.jpg`:

In [12]:
soup.find_all("img", src="/static/dionysus.jpg")

[<img src="/static/dionysus.jpg"/>]

Si pasamos algún tiempo navegando por varios sitios web y viendo sus HTML, notaremos que muchos sitios web tienen estructuras HTML extremadamente complicadas. En esos casos es en donde el anterior ejemplo tomará más valor.


En algunos casos, es posible que `BeautifulSoup` no ofrezca la funcionalidad que queremos. El paquete `lxml` es algo más complicado para comenzar, pero ofrece mucha más flexibilidad.


`BeautifulSoup` es excelente para extraer datos de un HTML, pero no proporciona ninguna forma de trabajar con formularios HTML.


## Interactuar con formularios HTML

El módulo `urllib` no proporciona un medio integrado para trabajar con páginas web de forma interactiva.

Existen otras opciones, en esta lección usaremos `MechanicalSoup` porque es relativamente sencillo de usar.

`MechanicalSoup` instala un *headless browser*, que es un navegador web sin interfaz gráfica. Este navegador se controla directamente con Python.

### Browser

La clase `Browser` representa el navegador web. Podemos usarlo para solicitar una página de Internet pasando una URL a su método `.get()`:

In [13]:
import mechanicalsoup
browser = mechanicalsoup.Browser()
url = "http://olympus.realpython.org/login"
page = browser.get(url)
page

<Response [200]>

`page` es un objeto `Response` que almacena la respuesta de la solicitud a la URL.

El número `200` representa el código de status devuelto por la solicitud. Un código de estado de `200` significa que la solicitud fue exitosa. 

Una solicitud fallida puede mostrar un código de estado de `404` si la URL no existe o `500` si hay un error del servidor al realizar la solicitud.

`MechanicalSoup` usa `BeautifulSoup` para analizar el HTML y por tanto `page` tiene el atributo `.soup` que representa un objeto `BeautifulSoup`:

In [14]:
page.soup

<html>
<head>
<title>Log In</title>
</head>
<body bgcolor="yellow">
<center>
<br/><br/>
<h2>Please log in to access Mount Olympus:</h2>
<br/><br/>
<form action="/login" method="post" name="login">
Username: <input name="user" type="text"/><br/>
Password: <input name="pwd" type="password"/><br/><br/>
<input type="submit" value="Submit"/>
</form>
</center>
</body>
</html>

Esta página tiene un tag `<form>` con elementos `<input>` para un nombre de usuario, una contraseña y para enviar la info.

### Enviar Formulario

Antes de continuar vamos a abrir la [página](http://olympus.realpython.org/login) en un navegador regular.

La sección del formulario es todo lo que se encuentra dentro de los tags `<form>` `</form>`. 

El formulario de esta página tiene el atributo de `name="login"`. 

Este formulario contiene dos elementos `<input>` uno con `name="user"` otro con `name="pwd"`.

El tercer elemento `<input>` es el botón *Submit*.

Ahora que tenemos clara la estructura podemos iniciar:

In [15]:
import mechanicalsoup

# 1 
browser = mechanicalsoup.Browser()
url = "http://olympus.realpython.org/login"
login_page = browser.get(url)

# 2
login_html = login_page.soup

# 3
form = login_html.select("form")[0]

# 4
form.select("input")[0]["value"] = "zeus"
form.select("input")[1]["value"] = "ThunderDude"

# 5
profiles_page = browser.submit(form, login_page.url)
profiles_page.url

'http://olympus.realpython.org/profiles'

Vamos a analizar cada paso

1. Creamos una instancia de la clase `Browser` y la usamos para solicitar la URL `http://olympus.realpython.org/login`. 
<br>

2. Asignamos el contenido HTML de la página a la variable `login_html` usando la propiedad `.soup`.
<br>

3. `login_html.select("formulario")` devuelve una lista de todos los elementos `<form>` de la página. Debido a que la página tiene solo un elemento `<form>`, podemos acceder al formulario recuperando el elemento en el índice 0 de la lista.
<br>

4. Las siguientes dos líneas seleccionan los tags `<input>` correspondientes al nombre de usuario y contraseña y les asignan los valores `"zeus"` y `"ThunderDude"`.
<br>

5. Enviamos el formulario con `browser.submit()`. El método `.submit()` acepta dos argumentos, el objeto `form` y la URL de `login_page`.

Ahora que hemos accedido a la página de perfiles, podemos acceder a las direcciones URL de todos los perfiles:

In [16]:
links = profiles_page.soup.select("a")
for link in links:
    endpoint = link["href"]
    name = link.text
    print(f"{name}: {endpoint}")

Aphrodite: /profiles/aphrodite
Poseidon: /profiles/poseidon
Dionysus: /profiles/dionysus


Las direcciones URL contenidas en cada atributo `"href"` son direcciones URL relativas. Así que debemos completarlas.

En este caso, la URL base es simplemente `http://olympus.realpython.org`. 

In [17]:
base_url = "http://olympus.realpython.org"
links = profiles_page.soup.select("a")
for link in links:
    endpoint = base_url + link["href"]
    name = link.text
    print(f"{name}: {endpoint}")

Aphrodite: http://olympus.realpython.org/profiles/aphrodite
Poseidon: http://olympus.realpython.org/profiles/poseidon
Dionysus: http://olympus.realpython.org/profiles/dionysus


Se pueden hacer muchas cosas con el paquete [`MechanicalSoup`](https://mechanicalsoup.readthedocs.io/en/stable/), les recomiendo mirar la documentación si les ha parecido interesante.